<a href="https://colab.research.google.com/github/Kagan777/Kaggle/blob/master/ga-customer-revenue-prediction/kaggle_ga_initial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# # Needed for json_normalize
# !pip install --upgrade pandas

    100% |████████████████████████████████| 8.9MB 3.5MB/s 
plotnine 0.5.1 has requirement matplotlib>=3.0.0, but you'll have matplotlib 2.1.2 which is incompatible.
  Found existing installation: pandas 0.22.0
    Uninstalling pandas-0.22.0:
      Successfully uninstalled pandas-0.22.0


In [7]:
ls

sample_submission_v2.csv.zip  train15.csv  train39.csv  train_v2.csv
test_v2.csv                   train23.csv  train47.csv  train_v2.csv.zip
test_v2.csv.zip               train31.csv  train7.csv


In [12]:
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
from ast import literal_eval
print(os.listdir("."))
import gc
gc.enable()

pd.options.display.max_columns = 999
# features = ['channelGrouping', 'date', 'fullVisitorId', 'visitId',\
#        'visitNumber', 'visitStartTime', 'device.browser',\
#        'device.deviceCategory', 'device.isMobile', 'device.operatingSystem',\
#        'geoNetwork.city', 'geoNetwork.continent', 'geoNetwork.country',\
#        'geoNetwork.metro', 'geoNetwork.networkDomain', 'geoNetwork.region',\
#        'geoNetwork.subContinent', 'totals.bounces', 'totals.hits',\
#        'totals.newVisits', 'totals.pageviews', 'totals.transactionRevenue',\
#        'trafficSource.adContent', 'trafficSource.campaign',\
#        'trafficSource.isTrueDirect', 'trafficSource.keyword',\
#        'trafficSource.medium', 'trafficSource.referralPath',\
#        'trafficSource.source']

features = ['appInfo.exitScreenName', 'appInfo.landingScreenName', 'channelGrouping',
            'contentGroup.contentGroup2', 'date', 'device_browser', 'device_deviceCategory',
            'device_isMobile', 'device_operatingSystem', 'eCommerceAction.action_type',
            'eventInfo.eventAction', 'eventInfo.eventCategory', 'fullVisitorId',
            'geoNetwork_city', 'geoNetwork_continent', 'geoNetwork_country',
            'geoNetwork_metro', 'geoNetwork_networkDomain', 'geoNetwork_region',
            'geoNetwork_subContinent', 'hitNumber', 'isEntrance', 'isExit',
            'page.hostname', 'page.pageTitle', 'productPrice', 'promoName',
            'promoPosition', 'promotionActionInfo.promoIsClick',
            'promotionActionInfo.promoIsView', 'referer', 'social.hasSocialSourceReferral', 
            'social.socialNetwork', 'totals_bounces', 'totals_hits',  'totals_newVisits',
            'totals_pageviews', 'totals_sessionQualityDim', 'totals_timeOnSite',
            'totals_transactionRevenue', 'totals_transactions', 'trafficSource_adContent',
            'trafficSource_adwordsClickInfo.adNetworkType', 'trafficSource_adwordsClickInfo.isVideoAd',
            'trafficSource_adwordsClickInfo.slot', 'trafficSource_campaign',
            'trafficSource_isTrueDirect', 'trafficSource_keyword', 'trafficSource_medium', 
            'trafficSource_referralPath', 'trafficSource_source', 'transaction.affiliation',
            'transaction.currencyCode', 'type', 'v2ProductCategory', 'v2ProductName',
            'totals_totalTransactionRevenue', 'visitId', 'visitNumber', 'visitStartTime']

def load_df(csv_path='train_v2.csv', start_chunk=0):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    ans = pd.DataFrame()
    dfs = pd.read_csv(csv_path, sep=',',
                      parse_dates=['date'],
                      converters={column: json.loads for column in JSON_COLUMNS}, 
                      dtype={'fullVisitorId': 'str'}, # Important!!
                      low_memory=False,
                      chunksize = 25000)
    for i, df in enumerate(dfs):
      if i >= start_chunk:
        df.reset_index(drop = True,inplace = True)
        if (i == start_chunk) and (start_chunk > 0):
          ans = pd.read_csv(f'train{i-1}.csv', sep=',' ,parse_dates=['date'], dtype={'fullVisitorId': 'str'}, low_memory=False)
        for column in JSON_COLUMNS:
            column_as_df = json_normalize(df[column])
            column_as_df.columns = [f"{column}_{subcolumn}" for subcolumn in column_as_df.columns]
            df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)

        df.loc[df[df['hits'] == "[]"].index, 'hits'] = "[{}]"
        df['hits'] = df['hits'].apply(literal_eval).str[0]
        column_as_df = json_normalize(df['hits'])
        df = df.drop('hits', axis=1).merge(column_as_df, right_index=True, left_index=True)

        # Extract the product and promo names from the complex nested structure into a simple flat list:
        if 'product' in df.columns:
            #print(chunk['product'][0])
            df['v2ProductName'] = df['product'].apply(lambda x: [p['v2ProductName'] for p in x] if type(x) == list else [])
            df['v2ProductCategory'] = df['product'].apply(lambda x: [p['v2ProductCategory'] for p in x] if type(x) == list else [])
            df['productPrice'] = df['product'].apply(lambda x: [p['productPrice'] for p in x] if type(x) == list else [])
            del df['product']
        if 'promotion' in df.columns:
            #print(chunk['promotion'][0])
            df['promoName']  = df['promotion'].apply(lambda x: [p['promoName'] for p in x] if type(x) == list else [])
            df['promoPosition']  = df['promotion'].apply(lambda x: [p['promoPosition'] for p in x] if type(x) == list else [])
            del df['promotion']        



        print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}, Chunk number: {i}")
        features_to_use = [x for x in df.columns if x in features]
        use_df = df[features_to_use]
        del df
        gc.collect()
        ans = pd.concat([ans, use_df], axis = 0).reset_index(drop = True)
        print(ans.shape)
        if (i+1) % 8 == 0:
          ans.to_csv(f'train{i}.csv')
      else:
        print(i)
    return ans

# train = load_df()
# train.shape

['train31.csv', 'train39.csv', 'test_v2.csv.zip', 'train_v2.csv', 'train23.csv', 'train_v2.csv.zip', 'sample_submission_v2.csv.zip', 'train47.csv', 'test_v2.csv', 'train15.csv', 'train7.csv']


In [52]:
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
from ast import literal_eval
print(os.listdir("."))
import gc
gc.enable()

pd.options.display.max_columns = 999
# features = ['channelGrouping', 'date', 'fullVisitorId', 'visitId',\
#        'visitNumber', 'visitStartTime', 'device.browser',\
#        'device.deviceCategory', 'device.isMobile', 'device.operatingSystem',\
#        'geoNetwork.city', 'geoNetwork.continent', 'geoNetwork.country',\
#        'geoNetwork.metro', 'geoNetwork.networkDomain', 'geoNetwork.region',\
#        'geoNetwork.subContinent', 'totals.bounces', 'totals.hits',\
#        'totals.newVisits', 'totals.pageviews', 'totals.transactionRevenue',\
#        'trafficSource.adContent', 'trafficSource.campaign',\
#        'trafficSource.isTrueDirect', 'trafficSource.keyword',\
#        'trafficSource.medium', 'trafficSource.referralPath',\
#        'trafficSource.source']

features = ['appInfo.exitScreenName', 'appInfo.landingScreenName', 'channelGrouping',
            'contentGroup.contentGroup2', 'date', 'device_browser', 'device_deviceCategory',
            'device_isMobile', 'device_operatingSystem', 'eCommerceAction.action_type',
            'eventInfo.eventAction', 'eventInfo.eventCategory', 'fullVisitorId',
            'geoNetwork_city', 'geoNetwork_continent', 'geoNetwork_country',
            'geoNetwork_metro', 'geoNetwork_networkDomain', 'geoNetwork_region',
            'geoNetwork_subContinent', 'hitNumber', 'isEntrance', 'isExit',
            'page.hostname', 'page.pageTitle', 'productPrice', 'promoName',
            'promoPosition', 'promotionActionInfo.promoIsClick',
            'promotionActionInfo.promoIsView', 'referer', 'social.hasSocialSourceReferral', 
            'social.socialNetwork', 'totals_bounces', 'totals_hits',  'totals_newVisits',
            'totals_pageviews', 'totals_sessionQualityDim', 'totals_timeOnSite',
            'totals_transactionRevenue', 'totals_transactions', 'trafficSource_adContent',
            'trafficSource_adwordsClickInfo.adNetworkType', 'trafficSource_adwordsClickInfo.isVideoAd',
            'trafficSource_adwordsClickInfo.slot', 'trafficSource_campaign',
            'trafficSource_isTrueDirect', 'trafficSource_keyword', 'trafficSource_medium', 
            'trafficSource_referralPath', 'trafficSource_source', 'transaction.affiliation',
            'transaction.currencyCode', 'type', 'v2ProductCategory', 'v2ProductName',
            'totals_totalTransactionRevenue', 'visitId', 'visitNumber', 'visitStartTime']

def load_df(csv_path='train_v2.csv', start_chunk=0):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    ans = pd.DataFrame()
    dfs = pd.read_csv(csv_path, sep=',',
                      parse_dates=['date'],
                      converters={column: json.loads for column in JSON_COLUMNS}, 
                      dtype={'fullVisitorId': 'str'}, # Important!!
                      low_memory=False,
                      chunksize = 50000)
    for i, df in enumerate(dfs):
      if i >= start_chunk:
        df.reset_index(drop = True,inplace = True)
        for column in JSON_COLUMNS:
            column_as_df = json_normalize(df[column])
            column_as_df.columns = [f"{column}_{subcolumn}" for subcolumn in column_as_df.columns]
            df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)

        df.loc[df[df['hits'] == "[]"].index, 'hits'] = "[{}]"
        df['hits'] = df['hits'].apply(literal_eval).str[0]
        column_as_df = json_normalize(df['hits'])
        df = df.drop('hits', axis=1).merge(column_as_df, right_index=True, left_index=True)

        # Extract the product and promo names from the complex nested structure into a simple flat list:
        if 'product' in df.columns:
            #print(chunk['product'][0])
            df['v2ProductName'] = df['product'].apply(lambda x: [p['v2ProductName'] for p in x] if type(x) == list else [])
            df['v2ProductCategory'] = df['product'].apply(lambda x: [p['v2ProductCategory'] for p in x] if type(x) == list else [])
            df['productPrice'] = df['product'].apply(lambda x: [p['productPrice'] for p in x] if type(x) == list else [])
            del df['product']
        if 'promotion' in df.columns:
            #print(chunk['promotion'][0])
            df['promoName']  = df['promotion'].apply(lambda x: [p['promoName'] for p in x] if type(x) == list else [])
            df['promoPosition']  = df['promotion'].apply(lambda x: [p['promoPosition'] for p in x] if type(x) == list else [])
            del df['promotion']        



        print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}, Chunk number: {i}")
        features_to_use = [x for x in df.columns if x in features]
        use_df = df[features_to_use]
        use_df.to_csv(f'clean_train{i}.csv')
        print(use_df.shape)        
        del df
        del use_df
        gc.collect()
#         ans = pd.concat([ans, use_df], axis = 0).reset_index(drop = True)

      else:
        print(i)
    return ans

# train = load_df()
# train.shape

['test_v2.csv.zip', 'train_v2.csv', 'backup_train47.csv', 'train_v2.csv.zip', 'sample_submission_v2.csv.zip', 'test_v2.csv']


In [0]:
cp clean* /content/gdrive/My\ Drive/kaggle/ga/tmp

In [61]:
ls

backup_train47.csv  clean_train21.csv  clean_train34.csv
clean_train0.csv    clean_train22.csv  clean_train3.csv
clean_train10.csv   clean_train23.csv  clean_train4.csv
clean_train11.csv   clean_train24.csv  clean_train5.csv
clean_train12.csv   clean_train25.csv  clean_train6.csv
clean_train13.csv   clean_train26.csv  clean_train7.csv
clean_train14.csv   clean_train27.csv  clean_train8.csv
clean_train15.csv   clean_train28.csv  clean_train9.csv
clean_train16.csv   clean_train29.csv  sample_submission_v2.csv.zip
clean_train17.csv   clean_train2.csv   test_v2.csv
clean_train18.csv   clean_train30.csv  test_v2.csv.zip
clean_train19.csv   clean_train31.csv  train_v2.csv
clean_train1.csv    clean_train32.csv  train_v2.csv.zip
clean_train20.csv   clean_train33.csv


In [0]:
tmp = pd.read_csv('train_v2.csv', usecols=['date'])

In [64]:
tmp.shape

(1708337, 1)

In [53]:
train = load_df()
train.shape

Loaded train_v2.csv. Shape: (50000, 118), Chunk number: 0
(50000, 59)
Loaded train_v2.csv. Shape: (50000, 129), Chunk number: 1
(50000, 59)
Loaded train_v2.csv. Shape: (50000, 130), Chunk number: 2
(50000, 59)
Loaded train_v2.csv. Shape: (50000, 129), Chunk number: 3
(50000, 59)
Loaded train_v2.csv. Shape: (50000, 129), Chunk number: 4
(50000, 59)
Loaded train_v2.csv. Shape: (50000, 129), Chunk number: 5
(50000, 59)
Loaded train_v2.csv. Shape: (50000, 127), Chunk number: 6
(50000, 59)
Loaded train_v2.csv. Shape: (50000, 129), Chunk number: 7
(50000, 59)
Loaded train_v2.csv. Shape: (50000, 129), Chunk number: 8
(50000, 59)
Loaded train_v2.csv. Shape: (50000, 129), Chunk number: 9
(50000, 59)
Loaded train_v2.csv. Shape: (50000, 135), Chunk number: 10
(50000, 60)
Loaded train_v2.csv. Shape: (50000, 117), Chunk number: 11
(50000, 59)
Loaded train_v2.csv. Shape: (50000, 129), Chunk number: 12
(50000, 59)
Loaded train_v2.csv. Shape: (50000, 129), Chunk number: 13
(50000, 59)
Loaded train_v2.

(0, 0)

['test_v2.csv.zip', 'train_v2.csv', 'train_v2.csv.zip', 'sample_submission_v2.csv.zip', 'test_v2.csv']
Loaded train_v2.csv. Shape: (25000, 118), Chunk number: 0
(25000, 59)
Loaded train_v2.csv. Shape: (25000, 118), Chunk number: 1
(50000, 59)
Loaded train_v2.csv. Shape: (25000, 129), Chunk number: 2
(75000, 59)
Loaded train_v2.csv. Shape: (25000, 118), Chunk number: 3
(100000, 59)
Loaded train_v2.csv. Shape: (25000, 129), Chunk number: 4
(125000, 59)
Loaded train_v2.csv. Shape: (25000, 128), Chunk number: 5
(150000, 59)
Loaded train_v2.csv. Shape: (25000, 128), Chunk number: 6
(175000, 59)
Loaded train_v2.csv. Shape: (25000, 128), Chunk number: 7
(200000, 59)
Loaded train_v2.csv. Shape: (25000, 118), Chunk number: 8
(225000, 59)
Loaded train_v2.csv. Shape: (25000, 127), Chunk number: 9


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:85: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



(250000, 59)
Loaded train_v2.csv. Shape: (25000, 128), Chunk number: 10
(275000, 59)
Loaded train_v2.csv. Shape: (25000, 129), Chunk number: 11
(300000, 59)
Loaded train_v2.csv. Shape: (25000, 126), Chunk number: 12
(325000, 59)
Loaded train_v2.csv. Shape: (25000, 126), Chunk number: 13
(350000, 59)
Loaded train_v2.csv. Shape: (25000, 127), Chunk number: 14
(375000, 59)
Loaded train_v2.csv. Shape: (25000, 128), Chunk number: 15
(400000, 59)
Loaded train_v2.csv. Shape: (25000, 129), Chunk number: 16
(425000, 59)
Loaded train_v2.csv. Shape: (25000, 118), Chunk number: 17
(450000, 59)
Loaded train_v2.csv. Shape: (25000, 129), Chunk number: 18
(475000, 59)
Loaded train_v2.csv. Shape: (25000, 129), Chunk number: 19
(500000, 59)
Loaded train_v2.csv. Shape: (25000, 135), Chunk number: 20
(525000, 60)
Loaded train_v2.csv. Shape: (25000, 127), Chunk number: 21
(550000, 60)
Loaded train_v2.csv. Shape: (25000, 117), Chunk number: 22
(575000, 60)
Loaded train_v2.csv. Shape: (25000, 117), Chunk num

In [15]:
tmp = pd.read_csv('train47.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8,13,21,29,53) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [42]:
cd -

/content/data


In [0]:
tmp = pd.read_csv('test_v2.csv', usecols=['visitId', 'date'], dtype = {'visitId':'str'})

In [45]:
tmp.shape

(401589, 2)

In [47]:
tmp1.shape

(401590, 60)

In [31]:
 cd ../gdrive/My\ Drive/kaggle/ga

/content/gdrive/My Drive/kaggle/ga


In [35]:
tmp1 = pd.read_csv('test.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7,12,20,52) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [36]:
tmp1.shape

(401590, 60)

In [37]:
tmp1

,appInfo.exitScreenName,appInfo.landingScreenName,channelGrouping,contentGroup.contentGroup2,date,device_browser,device_deviceCategory,device_isMobile,device_operatingSystem,eCommerceAction.action_type,eventInfo.eventAction,eventInfo.eventCategory,fullVisitorId,geoNetwork_city,geoNetwork_continent,geoNetwork_country,geoNetwork_metro,geoNetwork_networkDomain,geoNetwork_region,geoNetwork_subContinent,hitNumber,isEntrance,isExit,page.hostname,page.pageTitle,productPrice,promoName,promoPosition,promotionActionInfo.promoIsClick,promotionActionInfo.promoIsView,referer,social.hasSocialSourceReferral,social.socialNetwork,totals_bounces,totals_hits,totals_newVisits,totals_pageviews,totals_sessionQualityDim,totals_timeOnSite,totals_totalTransactionRevenue,totals_transactionRevenue,totals_transactions,trafficSource_adContent,trafficSource_adwordsClickInfo.adNetworkType,trafficSource_adwordsClickInfo.isVideoAd,trafficSource_adwordsClickInfo.slot,trafficSource_campaign,trafficSource_isTrueDirect,trafficSource_keyword,trafficSource_medium,trafficSource_referralPath,trafficSource_source,transaction.affiliation,transaction.currencyCode,type,v2ProductCategory,v2ProductName,visitId,visitNumber,visitStartTime
0,shop.googlemerchandisestore.com/home,shop.googlemerchandisestore.com/home,Organic Search,(not set),2018-05-11,Chrome,mobile,True,Android,0.0,Promotion Click,Enhanced Ecommerce,7460955084541987166,(not set),Asia,India,(not set),unknown.unknown,Delhi,Southern Asia,1,NaN,NaN,shop.googlemerchandisestore.com,Home,[],['Accessories'],['Category Row 2'],True,NaN,NaN,No,(not set),NaN,4.0,NaN,3.0,1.0,973.0,NaN,NaN,NaN,(not set),NaN,NaN,NaN,(not set),True,(not provided),organic,(not set),google,NaN,NaN,EVENT,[],[],1.526099e+09,2.0,1.526099e+09
1,shop.googlemerchandisestore.com/google+redesig...,shop.googlemerchandisestore.com/home,Direct,(not set),2018-05-11,Chrome,desktop,False,Macintosh,0.0,NaN,NaN,460252456180441002,San Francisco,Americas,United States,San Francisco-Oakland-San Jose CA,(not set),California,Northern America,1,True,NaN,shop.googlemerchandisestore.com,Home,[],"['Apparel', 'Backpacks', 'Mens T-Shirts', 'Wom...","['Row 1', 'Row 2 Combo', 'Row 3-1', 'Row 3-2',...",NaN,True,NaN,No,(not set),NaN,4.0,NaN,3.0,1.0,49.0,NaN,NaN,NaN,(not set),NaN,NaN,NaN,(not set),True,(not set),(none),(not set),(direct),NaN,NaN,PAGE,[],[],1.526064e+09,166.0,1.526064e+09
2,shop.googlemerchandisestore.com/google+redesig...,shop.googlemerchandisestore.com/google+redesig...,Organic Search,Apparel,2018-05-11,Chrome,desktop,False,Chrome OS,3.0,Add to Cart,Enhanced Ecommerce,3461808543879602873,not available in demo dataset,Americas,United States,not available in demo dataset,onlinecomputerworks.com,not available in demo dataset,Northern America,1,NaN,NaN,shop.googlemerchandisestore.com,Women's T-Shirts | Apparel | Google Merchandis...,['21990000'],[],[],NaN,NaN,https://www.google.com/,No,(not set),NaN,4.0,NaN,3.0,1.0,24.0,NaN,NaN,NaN,(not set),NaN,NaN,NaN,(not set),True,(not provided),organic,(not set),google,NaN,USD,EVENT,"[""Home/Apparel/Women's/Women's-T-Shirts/""]",['Google Tee White'],1.526067e+09,2.0,1.526067e+09
3,shop.googlemerchandisestore.com/google+redesig...,shop.googlemerchandisestore.com/home,Direct,(not set),2018-05-11,Chrome,mobile,True,iOS,0.0,NaN,NaN,975129477712150630,Houston,Americas,United States,Houston TX,(not set),Texas,Northern America,1,True,NaN,shop.googlemerchandisestore.com,Home,[],"['Apparel', 'Backpacks', 'Mens T-Shirts', 'Wom...","['Row 1', 'Row 2 Combo', 'Row 3-1', 'Row 3-2',...",NaN,True,https://www.googlemerchandisestore.com/,No,(not set),NaN,5.0,NaN,4.0,1.0,25.0,NaN,NaN,NaN,(not set),NaN,NaN,NaN,(not set),True,(not set),(none),(not set),(direct),NaN,NaN,PAGE,[],[],1.526108e+09,4.0,1.526108e+09
4,shop.googlemerchandisestore.com/asearch.html,shop.googlemerchandisestore.com/home,Organic Search,(not set),2018-05-11,Internet Explorer,tablet,True,Windows,0.0,NaN,NaN,8381672768065729990,Irvine,Americas,United States,Los Angeles CA,com,

In [19]:
tmp[tmp.index != tmp['Unnamed: 0']]

,Unnamed: 0,appInfo.exitScreenName,appInfo.landingScreenName,channelGrouping,contentGroup.contentGroup2,date,device_browser,device_deviceCategory,device_isMobile,device_operatingSystem,eCommerceAction.action_type,eventInfo.eventAction,eventInfo.eventCategory,fullVisitorId,geoNetwork_city,geoNetwork_continent,geoNetwork_country,geoNetwork_metro,geoNetwork_networkDomain,geoNetwork_region,geoNetwork_subContinent,hitNumber,isEntrance,isExit,page.hostname,page.pageTitle,productPrice,promoName,promoPosition,promotionActionInfo.promoIsClick,promotionActionInfo.promoIsView,referer,social.hasSocialSourceReferral,social.socialNetwork,totals_bounces,totals_hits,totals_newVisits,totals_pageviews,totals_sessionQualityDim,totals_timeOnSite,totals_totalTransactionRevenue,totals_transactionRevenue,totals_transactions,trafficSource_adContent,trafficSource_adwordsClickInfo.adNetworkType,trafficSource_adwordsClickInfo.isVideoAd,trafficSource_adwordsClickInfo.slot,trafficSource_campaign,trafficSource_isTrueDirect,trafficSource_keyword,trafficSource_medium,trafficSource_referralPath,trafficSource_source,transaction.affiliation,transaction.currencyCode,type,v2ProductCategory,v2ProductName,visitId,visitNumber,visitStartTime
65781,NaN,No,(not set),NaN,3,1,3,NaN,49,NaN,NaN,NaN,NaN,,NaN,NaN,(not set),NaN,NaN,referral,/,google.co.in,NaN,USD,PAGE,"[""Home/Apparel/Men's/Men's-T-Shirts/"", ""Home/A...","[""Google Men's 100% Cotton Short Sleeve Hero T...",1497181639,1,1497181639,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65782,65781.0,shop.googlemerchandisestore.com/google+redesig...,shop.googlemerchandisestore.com/google+redesig...,Organic Search,Brands,2017-06-11,Safari,mobile,True,iOS,0.0,NaN,NaN,4785286137375657853,not available in demo dataset,Asia,Saudi Arabia,not available in demo dataset,(not set),not available in demo dataset,Western Asia,1,True,NaN,shop.googlemerchandisestore.com,YouTube | Shop by Brand | Google Merchandise S...,"['7990000', '1990000', '6990000', '4990000', '...",[],[],NaN,NaN,https://www.google.co.uk/,No,(not set),NaN,3.0,1.0,3.0,NaN,31.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(not set),NaN,NaN,(none),NaN,(direct),NaN,USD,PAGE,"['Home/Shop by Brand/YouTube/', 'Home/Shop by ...","['24 oz YouTube Sergeant Stripe Bottle', 'YouT...",1.497204e+09,1.0,1.497204e+09
65783,65782.0,shop.googlemerchandisestore.com/google+redesig...,shop.googlemerchandisestore.com/google+redesig...,Organic Search,Brands,2017-06-11,Chrome,mobile,True,Android,0.0,NaN,NaN,2463010916201679521,not available in demo dataset,Asia,Indonesia,not available in demo dataset,unknown.unknown,not available in demo dataset,Southeast Asia,1,True,NaN,shop.googlemerchandisestore.com,YouTube | Shop by Brand | Google Merchandise S...,"['7990000', '1990000', '6990000', '4990000', '...",[],[],NaN,NaN,https://www.google.com/,No,(not set),NaN,3.0,1.0,3.0,NaN,199.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(not set),NaN,NaN,(none),NaN,(direct),NaN,USD,PAGE,"['Home/Shop by Brand/YouTube/', 'Home/Shop by ...","['24 oz YouTube Sergeant Stripe Bottle', 'YouT...",1.497238e+09,1.0,1.497238e+09
65784,65783.0,shop.googlemerchandisestore.com/google+redesig...,shop.googlemerchandisestore.com/google+redesig...,Organic Search,Brands,2017-06-11,Safari,mobile,True,iOS,0.0,NaN,NaN,7149546576463078106,not available in demo dataset,Americas,Canada,not available in demo dataset,mcsnet.ca,not available in demo dataset,Northern America,1,True,NaN,shop.googlemerchandisestore.com,YouTube | Shop by Brand | Google Merchandise S...,"['7990000', '1990000', '6990000', '4990000', '...",[],[],NaN,NaN,https://www.google.ca/,No,(not set),NaN,3.0,1.0,3.0,NaN,86.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(not set),NaN,NaN,(none),NaN,(direct),NaN,USD,PAGE,"['Home/Shop by Brand/YouTube/', 'Home/Shop by ...","['24 oz YouTube Sergeant Stripe Bottle', 'YouT...",1.497194e+09,1.0,1.497194e+09
65785,65784.0,shop.googlemerchandisestore.com/google+redesig...,shop.googlemerchandisesto

In [2]:
cd data

/content/data


In [3]:
ls

sample_submission_v2.csv.zip  train15.csv  train39.csv  train_v2.csv
test_v2.csv                   train23.csv  train47.csv  train_v2.csv.zip
test_v2.csv.zip               train31.csv  train7.csv


In [0]:
cp train47.csv ../gdrive/My\ Drive/kaggle/ga

In [13]:
test = load_df("test_v2.csv")

Loaded test_v2.csv. Shape: (25000, 126)
(25000, 59)
Loaded test_v2.csv. Shape: (25000, 129)
(50000, 59)
Loaded test_v2.csv. Shape: (25000, 138)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:85: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



(75000, 60)
Loaded test_v2.csv. Shape: (25000, 132)
(100000, 60)
Loaded test_v2.csv. Shape: (25000, 127)
(125000, 60)
Loaded test_v2.csv. Shape: (25000, 135)
(150000, 60)
Loaded test_v2.csv. Shape: (25000, 128)
(175000, 60)
Loaded test_v2.csv. Shape: (25000, 137)
(200000, 60)
Loaded test_v2.csv. Shape: (25000, 127)
(225000, 60)
Loaded test_v2.csv. Shape: (25000, 125)
(250000, 60)
Loaded test_v2.csv. Shape: (25000, 128)
(275000, 60)
Loaded test_v2.csv. Shape: (25000, 129)
(300000, 60)
Loaded test_v2.csv. Shape: (25000, 129)
(325000, 60)
Loaded test_v2.csv. Shape: (25000, 128)
(350000, 60)
Loaded test_v2.csv. Shape: (25000, 138)
(375000, 60)
Loaded test_v2.csv. Shape: (25000, 129)
(400000, 60)
Loaded test_v2.csv. Shape: (1589, 112)
(401589, 60)


In [2]:
cd gdrive/My\ Drive/kaggle/ga

/content/gdrive/My Drive/kaggle/ga


In [2]:
# cd ../gdrive/My\ Drive/kaggle/ga

[Errno 2] No such file or directory: '../gdrive/My Drive/kaggle/ga'
/content


In [21]:
  dfs = pd.read_csv("test_v2.csv", sep=',',
                    parse_dates=['date'],
                    #converters={column: json.loads for column in JSON_COLUMNS}, 
                    dtype={'fullVisitorId': 'str'}, # Important!!
                    low_memory=False,
                    chunksize = 25000)
  for i, df in enumerate(dfs):
    print (i)

0
1
2
3
4
5


KeyboardInterrupt: ignored

In [18]:
cd -

/content/data


In [0]:
test.to_csv("test.csv", index=False)

In [0]:
test = pd.read_csv('test_pre_clean.csv', low_memory=False)

In [7]:
test.get[]

,appInfo.exitScreenName,appInfo.landingScreenName,appInfo.screenDepth,appInfo.screenName,channelGrouping,contentGroup.contentGroup1,contentGroup.contentGroup2,contentGroup.contentGroup3,contentGroup.contentGroup4,contentGroup.contentGroup5,contentGroup.contentGroupUniqueViews1,contentGroup.contentGroupUniqueViews2,contentGroup.contentGroupUniqueViews3,contentGroup.previousContentGroup1,contentGroup.previousContentGroup2,contentGroup.previousContentGroup3,contentGroup.previousContentGroup4,contentGroup.previousContentGroup5,customDimensions_x,customDimensions_y,customMetrics,customVariables,dataSource,date,device_browser,device_browserSize,device_browserVersion,device_deviceCategory,device_flashVersion,device_isMobile,device_language,device_mobileDeviceBranding,device_mobileDeviceInfo,device_mobileDeviceMarketingName,device_mobileDeviceModel,device_mobileInputSelector,device_operatingSystem,device_operatingSystemVersion,device_screenColors,device_screenResolution,eCommerceAction.action_type,eCommerceAction.option,eCommerceAction.step,eventInfo.eventAction,eventInfo.eventCategory,eventInfo.eventLabel,exceptionInfo.isFatal,experiment,fullVisitorId,geoNetwork_city,geoNetwork_cityId,geoNetwork_continent,geoNetwork_country,geoNetwork_latitude,geoNetwork_longitude,geoNetwork_metro,geoNetwork_networkDomain,geoNetwork_networkLocation,geoNetwork_region,geoNetwork_subContinent,hitNumber,hour,isEntrance,isExit,isInteraction,item.currencyCode,item.transactionId,latencyTracking.domContentLoadedTime,latencyTracking.domInteractiveTime,latencyTracking.domLatencyMetricsSample,latencyTracking.domainLookupTime,latencyTracking.pageDownloadTime,latencyTracking.pageLoadSample,latencyTracking.pageLoadTime,latencyTracking.redirectionTime,latencyTracking.serverConnectionTime,latencyTracking.serverResponseTime,latencyTracking.speedMetricsSample,minute,page.hostname,page.pagePath,page.pagePathLevel1,page.pagePathLevel2,page.pagePathLevel3,page.pagePathLevel4,page.pageTitle,page.searchCategory,page.searchKeyword,productPrice,promoName,promoPosition,promotionActionInfo.promoIsClick,promotionActionInfo.promoIsView,publisher_infos,referer,social.hasSocialSourceReferral,social.socialInteractionNetworkAction,social.socialNetwork,socialEngagementType,time,totals_bounces,totals_hits,totals_newVisits,totals_pageviews,totals_sessionQualityDim,totals_timeOnSite,totals_totalTransactionRevenue,totals_transactionRevenue,totals_transactions,totals_visits,trafficSource_adContent,trafficSource_adwordsClickInfo.adNetworkType,trafficSource_adwordsClickInfo.criteriaParameters,trafficSource_adwordsClickInfo.gclId,trafficSource_adwordsClickInfo.isVideoAd,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.slot,trafficSource_campaign,trafficSource_isTrueDirect,trafficSource_keyword,trafficSource_medium,trafficSource_referralPath,trafficSource_source,transaction.affiliation,transaction.currencyCode,transaction.localTransactionRevenue,transaction.localTransactionShipping,transaction.localTransactionTax,transaction.transactionId,transaction.transactionRevenue,transaction.transactionShipping,transaction.transactionTax,type,v2ProductCategory,v2ProductName,visitId,visitNumber,visitStartTime
0,shop.googlemerchandisestore.com/home,shop.googlemerchandisestore.com/home,0,shop.googlemerchandisestore.com/home,Organic Search,(not set),(not set),(not set),(not set),(not set),NaN,NaN,NaN,(entrance),(entrance),(entrance),(entrance),(entrance),"[{'index': '4', 'value': 'APAC'}]",[],[],[],web,20180511,Chrome,not available in demo dataset,not available in demo dataset,mobile,not available in demo dataset,True,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Android,not available in demo dataset,not available in demo dataset,not available in demo dataset,0,NaN,1.0,Promotion Click,Enhanced Ecommerce,NaN,True,[],7460955084541987166,(not set),not available

In [171]:
all_columns_to_keep = sorted(list(set(columns_to_keep + columns_to_keep1 + columns_to_keep2 + columns_to_keep3 + updated_columns6)))
all_columns_to_keep

['appInfo.exitScreenName',
 'appInfo.landingScreenName',
 'channelGrouping',
 'contentGroup.contentGroup2',
 'date',
 'device_browser',
 'device_deviceCategory',
 'device_isMobile',
 'device_operatingSystem',
 'eCommerceAction.action_type',
 'eventInfo.eventAction',
 'eventInfo.eventCategory',
 'fullVisitorId',
 'geoNetwork_city',
 'geoNetwork_continent',
 'geoNetwork_country',
 'geoNetwork_metro',
 'geoNetwork_networkDomain',
 'geoNetwork_region',
 'geoNetwork_subContinent',
 'hitNumber',
 'isEntrance',
 'isExit',
 'page.hostname',
 'page.pageTitle',
 'productPrice',
 'promoName',
 'promoPosition',
 'promotionActionInfo.promoIsClick',
 'promotionActionInfo.promoIsView',
 'referer',
 'social.hasSocialSourceReferral',
 'social.socialNetwork',
 'totals_bounces',
 'totals_hits',
 'totals_newVisits',
 'totals_pageviews',
 'totals_sessionQualityDim',
 'totals_timeOnSite',
 'totals_transactionRevenue',
 'totals_transactions',
 'trafficSource_adContent',
 'trafficSource_adwordsClickInfo.adNet

In [14]:
type(test.iloc[0,20])

str

In [169]:
len(set(all_columns_to_keep))

59

In [0]:
list_columns = []
for col in test.columns:
  if isinstance(test.loc[0, col], list):
    list_columns.append(col)

In [16]:
list_columns

[]

In [78]:
list_columns

['customDimensions_y',
 'customMetrics',
 'customVariables',
 'experiment',
 'productPrice',
 'promoName',
 'promoPosition',
 'publisher_infos',
 'v2ProductCategory',
 'v2ProductName']

In [20]:
test.head()

,appInfo.exitScreenName,appInfo.landingScreenName,appInfo.screenDepth,appInfo.screenName,channelGrouping,contentGroup.contentGroup1,contentGroup.contentGroup2,contentGroup.contentGroup3,contentGroup.contentGroup4,contentGroup.contentGroup5,contentGroup.contentGroupUniqueViews1,contentGroup.contentGroupUniqueViews2,contentGroup.contentGroupUniqueViews3,contentGroup.previousContentGroup1,contentGroup.previousContentGroup2,contentGroup.previousContentGroup3,contentGroup.previousContentGroup4,contentGroup.previousContentGroup5,customDimensions_x,customDimensions_y,customMetrics,customVariables,dataSource,date,device_browser,device_browserSize,device_browserVersion,device_deviceCategory,device_flashVersion,device_isMobile,device_language,device_mobileDeviceBranding,device_mobileDeviceInfo,device_mobileDeviceMarketingName,device_mobileDeviceModel,device_mobileInputSelector,device_operatingSystem,device_operatingSystemVersion,device_screenColors,device_screenResolution,eCommerceAction.action_type,eCommerceAction.option,eCommerceAction.step,eventInfo.eventAction,eventInfo.eventCategory,eventInfo.eventLabel,exceptionInfo.isFatal,experiment,fullVisitorId,geoNetwork_city,geoNetwork_cityId,geoNetwork_continent,geoNetwork_country,geoNetwork_latitude,geoNetwork_longitude,geoNetwork_metro,geoNetwork_networkDomain,geoNetwork_networkLocation,geoNetwork_region,geoNetwork_subContinent,hitNumber,hour,isEntrance,isExit,isInteraction,item.currencyCode,item.transactionId,latencyTracking.domContentLoadedTime,latencyTracking.domInteractiveTime,latencyTracking.domLatencyMetricsSample,latencyTracking.domainLookupTime,latencyTracking.pageDownloadTime,latencyTracking.pageLoadSample,latencyTracking.pageLoadTime,latencyTracking.redirectionTime,latencyTracking.serverConnectionTime,latencyTracking.serverResponseTime,latencyTracking.speedMetricsSample,minute,page.hostname,page.pagePath,page.pagePathLevel1,page.pagePathLevel2,page.pagePathLevel3,page.pagePathLevel4,page.pageTitle,page.searchCategory,page.searchKeyword,productPrice,promoName,promoPosition,promotionActionInfo.promoIsClick,promotionActionInfo.promoIsView,publisher_infos,referer,social.hasSocialSourceReferral,social.socialInteractionNetworkAction,social.socialNetwork,socialEngagementType,time,totals_bounces,totals_hits,totals_newVisits,totals_pageviews,totals_sessionQualityDim,totals_timeOnSite,totals_totalTransactionRevenue,totals_transactionRevenue,totals_transactions,totals_visits,trafficSource_adContent,trafficSource_adwordsClickInfo.adNetworkType,trafficSource_adwordsClickInfo.criteriaParameters,trafficSource_adwordsClickInfo.gclId,trafficSource_adwordsClickInfo.isVideoAd,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.slot,trafficSource_campaign,trafficSource_isTrueDirect,trafficSource_keyword,trafficSource_medium,trafficSource_referralPath,trafficSource_source,transaction.affiliation,transaction.currencyCode,transaction.localTransactionRevenue,transaction.localTransactionShipping,transaction.localTransactionTax,transaction.transactionId,transaction.transactionRevenue,transaction.transactionShipping,transaction.transactionTax,type,v2ProductCategory,v2ProductName,visitId,visitNumber,visitStartTime
0,shop.googlemerchandisestore.com/home,shop.googlemerchandisestore.com/home,0,shop.googlemerchandisestore.com/home,Organic Search,(not set),(not set),(not set),(not set),(not set),NaN,NaN,NaN,(entrance),(entrance),(entrance),(entrance),(entrance),"[{'index': '4', 'value': 'APAC'}]",[],[],[],web,20180511,Chrome,not available in demo dataset,not available in demo dataset,mobile,not available in demo dataset,True,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Android,not available in demo dataset,not available in demo dataset,not available in demo dataset,0,NaN,1.0,Promotion Click,Enhanced Ecommerce,NaN,True,[],7460955084541987166,(not set),not available

In [1]:
test[list_columns]

NameError: ignored

In [0]:
not_list_columns = [x for x in test.columns if x not in list_columns]

In [0]:
tmp = [(test[not_list_columns].isnull().sum(axis=0) == 0) & (test[not_list_columns].nunique() == 1)]
columns_with_fixed_value = tmp[0][tmp[0] == True].index

In [19]:
columns_with_fixed_value

Index([], dtype='object')

In [0]:
updated_columns = [x for x in not_list_columns if x not in columns_with_fixed_value]

In [110]:
test.shape

(401589, 138)

In [0]:
tmp = test.nunique() == 1

In [39]:
test[tmp[tmp].index].apply(pd.Series.value_counts, dropna=False)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexes/api.py:107: RuntimeWarning: '<' not supported between instances of 'float' and 'str', sort order is undefined for incomparable objects
  result = result.union(other)
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:7397: RuntimeWarning: '<' not supported between instances of 'str' and 'float', sort order is undefined for incomparable objects
  index = _union_indexes(indexes)
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:7397: RuntimeWarning: '<' not supported between instances of 'float' and 'str', sort order is undefined for incomparable objects
  index = _union_indexes(indexes)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexes/api.py:107: RuntimeWarning: '<' not supported between instances of 'str' and 'float', sort order is undefined for incomparable objects
  result = result.union(other)


,contentGroup.contentGroupUniqueViews1,contentGroup.contentGroupUniqueViews2,contentGroup.contentGroupUniqueViews3,contentGroup.previousContentGroup1,contentGroup.previousContentGroup2,contentGroup.previousContentGroup5,customDimensions_y,customMetrics,customVariables,device_mobileDeviceBranding,device_mobileDeviceInfo,device_mobileDeviceMarketingName,device_mobileDeviceModel,device_mobileInputSelector,device_operatingSystemVersion,exceptionInfo.isFatal,experiment,geoNetwork_cityId,geoNetwork_latitude,geoNetwork_longitude,geoNetwork_networkLocation,isEntrance,isExit,isInteraction,item.currencyCode,latencyTracking.domLatencyMetricsSample,latencyTracking.pageLoadSample,latencyTracking.speedMetricsSample,page.searchCategory,promotionActionInfo.promoIsClick,promotionActionInfo.promoIsView,publisher_infos,social.socialInteractionNetworkAction,socialEngagementType,time,totals_bounces,totals_newVisits,totals_visits,trafficSource_adwordsClickInfo.criteriaParameters,trafficSource_adwordsClickInfo.isVideoAd,trafficSource_isTrueDirect,transaction.affiliation,transaction.currencyCode
NaN,399465.0,249061.0,401541.0,59.0,59.0,59.0,59.0,59.0,59.0,1.0,1.0,1.0,1.0,1.0,1.0,59.0,59.0,1.0,1.0,1.0,1.0,2080.0,218064.0,59.0,239308.0,401537.0,401537.0,401537.0,401475.0,400962.0,299652.0,59.0,60.0,2.0,60.0,218913.0,115526.0,2.0,2.0,390985.0,253181.0,401584.0,239309.0
1.0,2125.0,152529.0,49.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,401531.0,NaN,NaN,NaN,NaN,NaN,399510.0,183526.0,401531.0,NaN,53.0,53.0,53.0,NaN,628.0,101938.0,NaN,NaN,NaN,NaN,182677.0,286064.0,401588.0,NaN,NaN,148409.0,NaN,NaN
(entrance),NaN,NaN,NaN,401531.0,401531.0,401531.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
[],NaN,NaN,NaN,NaN,NaN,NaN,401531.0,401531.0,401531.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,401531.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,401531.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,401589.0,401589.0,401589.0,401589.0,401589.0,401589.0,NaN,NaN,401589.0,401589.0,401589.0,401589.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,401588.0,NaN,NaN,NaN,NaN
USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,162282.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,162281.0
(not set),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,115.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,401530.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Not Socially Engaged,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,401588.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,401530.0,NaN,NaN,NaN,NaN,10605.0,NaN,NaN,NaN


In [0]:
columns_to_keep1 = ['isEntrance'
,'isExit'
,'promotionActionInfo.promoIsClick'
,'promotionActionInfo.promoIsView'
,'totals_bounces'
,'totals_newVisits'
,'trafficSource_adwordsClickInfo.isVideoAd'
,'trafficSource_isTrueDirect'
,'transaction.affiliation'
,'transaction.currencyCode']



In [0]:
columns_with_one_nunique = [x for x in list(tmp[tmp].index)]
updated_columns1 = [x for x in test.columns if x not in columns_with_one_nunique]

In [76]:
test.shape

(401590, 138)

In [75]:
test[updated_columns1].shape

(401590, 95)

In [0]:
tmp = test[updated_columns1].nunique() <= 2

In [0]:
# test[tmp[tmp].index].apply(pd.Series.value_counts, dropna=False)

In [0]:
# test[test.totals_transactionRevenue > 0][tmp[tmp].index].apply(pd.Series.value_counts, dropna=False)

In [0]:
columns_to_keep2 = ['device_isMobile'
,'eventInfo.eventCategory'
,'page.hostname'
,'social.hasSocialSourceReferral'
,'trafficSource_adwordsClickInfo.adNetworkType'
,'type']

In [0]:
columns_with_two_nunique = [x for x in list(tmp[tmp].index)]
updated_columns2 = [x for x in updated_columns1 if x not in columns_with_two_nunique]

In [83]:
test[updated_columns2].shape

(401590, 76)

In [0]:
tmp = test[updated_columns2].nunique() <= 5

In [0]:
# test[tmp[tmp].index].apply(pd.Series.value_counts, dropna=False)

In [0]:
# test[test.totals_transactionRevenue > 0][tmp[tmp].index].apply(pd.Series.value_counts, dropna=False)

In [0]:
columns_to_keep3 = ['device_deviceCategory'
,'trafficSource_adwordsClickInfo.slot']

In [0]:
columns_with_three_to_five_nunique = [x for x in list(tmp[tmp].index)]
updated_columns3 = [x for x in updated_columns2 if x not in columns_with_three_to_five_nunique]

In [93]:
test[updated_columns3].shape

(401590, 64)

In [0]:
columns_to_keep = ['channelGrouping', 'date', 'fullVisitorId', 'visitId',\
       'visitNumber', 'visitStartTime', 'device_browser',\
       'device_deviceCategory', 'device_isMobile', 'device_operatingSystem',\
       'geoNetwork_city', 'geoNetwork_continent', 'geoNetwork_country',\
       'geoNetwork_metro', 'geoNetwork_networkDomain', 'geoNetwork_region',\
       'geoNetwork_subContinent', 'totals_bounces', 'totals_hits',\
       'totals_newVisits', 'totals_pageviews', 'totals_transactionRevenue',\
       'trafficSource_adContent', 'trafficSource_campaign',\
       'trafficSource_isTrueDirect', 'trafficSource_keyword',\
       'trafficSource_medium', 'trafficSource_referralPath',\
       'trafficSource_source', 'productPrice', 'promoName', 'promoPosition',\
       'v2ProductCategory', 'v2ProductName']

updated_columns4 = [x for x in updated_columns3 if x not in columns_to_keep]

In [106]:
test[updated_columns4].shape

(401590, 35)

In [0]:
columns_to_discard1 = [x for x in list(test[updated_columns4]) if x.startswith('latencyTracking')] + ['hour', 'minute']
updated_columns5 = [x for x in updated_columns4 if x not in columns_to_discard1]

In [117]:
test[updated_columns5].shape

(401590, 25)

In [123]:
test[updated_columns5].isnull().sum()

appInfo.exitScreenName                      59
appInfo.landingScreenName                   58
appInfo.screenName                          58
contentGroup.contentGroup2                  58
customDimensions_x                           0
eCommerceAction.action_type                 58
eventInfo.eventAction                   399621
eventInfo.eventLabel                    400709
hitNumber                                   59
item.transactionId                      401584
page.pagePath                               59
page.pagePathLevel1                         59
page.pagePathLevel2                     240863
page.pagePathLevel3                     273723
page.pagePathLevel4                     371851
page.pageTitle                              81
page.searchKeyword                      401475
referer                                 115855
social.socialNetwork                        60
totals_sessionQualityDim                     2
totals_timeOnSite                       183687
totals_totalT

In [0]:
columns_to_discard2 = ['totals_totalTransactionRevenue', 'customDimensions_x', 'appInfo.screenName', 'eventInfo.eventLabel', 
                       'item.transactionId', 'page.searchKeyword', 'page.pagePath', 'page.pagePathLevel1', 'page.pagePathLevel2', 
                       'page.pagePathLevel3', 'page.pagePathLevel4', 'trafficSource_adwordsClickInfo.gclId', 'transaction.transactionId']
updated_columns6 = [x for x in updated_columns5 if x not in columns_to_discard2]

In [163]:
updated_columns6

['appInfo.exitScreenName',
 'appInfo.landingScreenName',
 'contentGroup.contentGroup2',
 'eCommerceAction.action_type',
 'eventInfo.eventAction',
 'hitNumber',
 'page.pageTitle',
 'referer',
 'social.socialNetwork',
 'totals_sessionQualityDim',
 'totals_timeOnSite',
 'totals_transactions']

In [0]:
# test[test['appInfo.landingScreenName'] != test['appInfo.screenName']][['appInfo.exitScreenName', 'appInfo.landingScreenName', 'appInfo.screenName']]

In [161]:
test['totals_timeOnSite'].value_counts()

10.0      2242
7.0       2205
9.0       2186
8.0       2185
6.0       2176
5.0       2148
11.0      2131
13.0      2112
12.0      1965
4.0       1955
14.0      1954
15.0      1905
16.0      1876
19.0      1854
17.0      1852
21.0      1806
18.0      1805
20.0      1732
22.0      1697
24.0      1683
25.0      1654
23.0      1641
29.0      1566
26.0      1557
27.0      1541
28.0      1515
32.0      1505
30.0      1498
3.0       1464
34.0      1421
          ... 
2146.0       1
3452.0       1
3078.0       1
3767.0       1
3421.0       1
4859.0       1
2427.0       1
2370.0       1
2929.0       1
4851.0       1
2425.0       1
2867.0       1
4845.0       1
3632.0       1
3204.0       1
7557.0       1
2876.0       1
3856.0       1
2417.0       1
4833.0       1
3783.0       1
4574.0       1
3270.0       1
4829.0       1
4540.0       1
3787.0       1
3488.0       1
4823.0       1
2879.0       1
3114.0       1
Name: totals_timeOnSite, Length: 3578, dtype: int64

In [157]:
test[test.totals_transactionRevenue > 0]['transaction.transactionId'].value_counts()

ORD201809261449    1
ORD201809211179    1
ORD201808031131    1
ORD201807071152    1
ORD20180817999     1
Name: transaction.transactionId, dtype: int64

In [181]:
test['transaction.affiliation'].value_counts()

Google Merchandise Store    6
Name: transaction.affiliation, dtype: int64

In [0]:
cd -

In [0]:
# gdrive/My\ Drive/kaggle/ga
test.to_csv("test.csv", index=False)

In [5]:
# train.to_csv("train.csv", index=False)
# test.to_csv("test.csv", index = False)
# train.head()

,channelGrouping,date,fullVisitorId,visitId,visitNumber,visitStartTime,device.browser,device.deviceCategory,device.isMobile,device.operatingSystem,geoNetwork.city,geoNetwork.continent,geoNetwork.country,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.region,geoNetwork.subContinent,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.transactionRevenue,trafficSource.adContent,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,Organic Search,20171016,3162355547410993243,1508198450,1,1508198450,Firefox,desktop,False,Windows,not available in demo dataset,Europe,Germany,not available in demo dataset,(not set),not available in demo dataset,Western Europe,1,1,1,1,NaN,NaN,(not set),NaN,water bottle,organic,NaN,google
1,Referral,20171016,8934116514970143966,1508176307,6,1508176307,Chrome,desktop,False,Chrome OS,Cupertino,Americas,United States,San Francisco-Oakland-San Jose CA,(not set),California,Northern America,NaN,2,NaN,2,NaN,NaN,(not set),NaN,NaN,referral,/a/google.com/transportation/mtv-services/bike...,sites.google.com
2,Direct,20171016,7992466427990357681,1508201613,1,1508201613,Chrome,mobile,True,Android,not available in demo dataset,Americas,United States,not available in demo dataset,windjammercable.net,not available in demo dataset,Northern America,NaN,2,1,2,NaN,NaN,(not set),True,NaN,(none),NaN,(direct)
3,Organic Search,20171016,9075655783635761930,1508169851,1,1508169851,Chrome,desktop,False,Windows,not available in demo dataset,Asia,Turkey,not available in demo dataset,unknown.unknown,not available in demo dataset,Western Asia,NaN,2,1,2,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
4,Organic Search,20171016,6960673291025684308,1508190552,1,1508190552,Chrome,desktop,False,Windows,not available in demo dataset,Americas,Mexico,not available in demo dataset,prod-infinitum.com.mx,not available in demo dataset,Central America,NaN,2,1,2,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google


In [0]:
# from google.colab import files
# files.download('test.csv') 

In [14]:
# from google.colab import drive
# drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!mkdir ../../../../data

In [9]:
cd ../../../../data/

/content/data


In [33]:
cd ../../../../data/ga/

/content/data/ga


In [22]:
ls

sample_submission_v2.csv.zip  test_v2.csv  test_v2.csv.zip  train_v2.csv.zip


In [0]:
!mkdir data

In [9]:
cd data

/content/data


In [10]:
!kaggle competitions download -c ga-customer-revenue-prediction 

  0% 0.00/2.55M [00:00<?, ?B/s]
100% 2.55M/2.55M [00:00<00:00, 126MB/s]
 95% 289M/304M [00:07<00:00, 23.7MB/s]
100% 304M/304M [00:07<00:00, 42.5MB/s]
 99% 1.02G/1.03G [00:25<00:00, 47.8MB/s]
100% 1.03G/1.03G [00:25<00:00, 42.9MB/s]


In [0]:
!unzip -qq train_v2.csv.zip

In [0]:
!unzip -qq test_v2.csv.zip

In [0]:
import pandas as pd

In [6]:
pd.__version__

'0.23.4'

In [0]:
JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
# ans = pd.DataFrame()
# dfs = pd.read_csv('train_v2.csv', sep=',',
#                  converters={column: json.loads for column in JSON_COLUMNS}, 
#                  dtype={'fullVisitorId': 'str'}, # Important!!
#                 chunksize = 100000)

In [0]:
reader = pd.read_csv('train_v2.csv', sep=',',
                 converters={column: json.loads for column in JSON_COLUMNS}, 
                 dtype={'fullVisitorId': 'str'}, # Important!!
                 iterator=True)
df = reader.get_chunk(10**4)

In [0]:
df = reader.get_chunk(10**4)

In [0]:
chunk = df.hits.copy()

In [0]:
chunk[chunk == '[]'] = "[{}]"

In [0]:
from ast import literal_eval

In [11]:
 %time chunk_litral = chunk.apply(literal_eval).str[0]

CPU times: user 41.2 s, sys: 1.03 s, total: 42.2 s
Wall time: 42.3 s


In [18]:
%time chunk_processed = json_normalize(chunk_litral)

CPU times: user 4.12 s, sys: 8.98 ms, total: 4.13 s
Wall time: 4.14 s


In [0]:
if 'product' in chunk_processed.columns:
    #print(chunk['product'][0])
    chunk_processed['v2ProductName'] = chunk_processed['product'].apply(lambda x: [p['v2ProductName'] for p in x] if type(x) == list else [])
    chunk_processed['v2ProductCategory'] = chunk_processed['product'].apply(lambda x: [p['v2ProductCategory'] for p in x] if type(x) == list else [])
    chunk_processed['productPrice'] = chunk_processed['product'].apply(lambda x: [p['productPrice'] for p in x] if type(x) == list else [])
#     del chunk['product']
if 'promotion' in chunk_processed.columns:
    #print(chunk['promotion'][0])
    chunk_processed['promoName']  = chunk_processed['promotion'].apply(lambda x: [p['promoName'] for p in x] if type(x) == list else [])
    chunk_processed['promoPosition']  = chunk_processed['promotion'].apply(lambda x: [p['promoPosition'] for p in x] if type(x) == list else [])
#     del chunk['promotion']

In [24]:
chunk_processed.shape

(10000, 60)

In [5]:
for i, df in enumerate(dfs):
    df.reset_index(drop = True,inplace = True)
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    if i == 0:
      break

JSONDecodeError: ignored

In [39]:
df.shape

(100000, 59)